In [100]:
# Owner: Amitabh Bhattacharya
# email: a.bhattacharya@unf.edu

# Importing necessary packages
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from functools import reduce

import sklearn.metrics as met
warnings.simplefilter(action = 'ignore')


In [101]:
# Quinlan’s ID3 and C4.5 decision tree learning algorithm

# The class is the node of a decision tree. It can be a root, intermediate node or a leaf node.
# Leaf node means it is the label
class Node:    
    # Constructor
    def __init__(self, col=None, dec=None):
        # Either col or dec should be None (Internal node or leaf)
        self.name = col
        self.decision = dec
        self.branches = []
        self.branch_names = []
        
    # This function is used to do post-pruning of the trained decision tree
    # The method is not implemented in this version
    def prune():
        pass

    # This function is used to predict the test dataset.
    def predict(self, df):
        prediction = []
        
        for ind, row in df.iterrows():
            result = self.row_predict(row)
            prediction.append(result)
            
        return pd.Series(prediction)        
    
    def row_predict(self, row):
        pred = ""
        if self.name != None:
            att = self.name
            att_val = row[att]
            att_ind = self.branch_names.index(att_val)
            att_br_tree = self.branches[att_ind]

            pred = att_br_tree.row_predict(row)            
        else:
            return self.decision
        
        return pred
                

    # The below function is to print the decision tree. 
    # It prints internal node/leaf with the information about the level it belongs.
    def print(self, name="Root", lev=0):       
        if self.name != None:
            print(lev*"   " + "-(" + name + ")-> " + "N: ", self.name)           
        
            for i in range(len(self.branches)):
                n = self.branch_names[i]
                b = self.branches[i]
                b.print(n, lev+1)
        else:
            print(lev*"   " + "-(" + name + ")-> " + "L: ", self.decision)
    
# This function calculates entropy of a column w.r.t. label (y)
def entropy(label):
    num_unique = label.nunique() 
    entropy_label = 0
    
    for i in range(num_unique):
        e_vc = label.value_counts()[i]/len(label)
        entropy_label += -1 * e_vc * math.log2(e_vc)

    return entropy_label

# This function calculates the information gain for branching on the column
def info_gain(current_ent, col, df):
    information = 0
    vc = df[col].value_counts()
        
    for i in range(len(vc)):
        p = vc[i]/len(df)  # probality of a particular class in the attribute
        # Entropy of the column with the particular class in the attribute
        s = entropy(df[df[col] == vc.index[i]].iloc[:,0])
        temp_ent = p * s
        information += temp_ent
        
    return round((current_ent - information), 3)

# This function calculates the gain ratio for branching on the column
def gain_ratio(current_ent, col, df):
    information = 0
    vc = df[col].value_counts()
        
    for i in range(len(vc)):
        p = vc[i]/len(df)  # probality of a particular class in the attribute
        # Entropy of the column with the particular class in the attribute
        s = entropy(df[df[col] == vc.index[i]].iloc[:,0])
        temp_ent = p * s
        information += temp_ent
        
    gain = current_ent - information
    
    # Computing the intrinsic value
    iv = 0    
    for i in range(len(vc)):
        iv += -1 * (vc[i]/len(df)) * math.log2(vc[i]/len(df))
    
    # Gain ratio - gain/intrinsic value 
    if iv != 0:
        gain_ratio = gain/iv
    else:
        gain_ratio = 0        
    
    return round(gain_ratio, 3)


# This assumes that the the first column in df input is the y label
# algorithm = 'id3' for information gain and algorithm = 'c4.5' for gain ratio
def fit_dt(df, algorithm = 'id3'):
    algo = algorithm
    columns = df.columns[1:]  # 0th column is the y label    
    current_ent = entropy(df.iloc[:,0]) # Obtaining the current entropy before spliting it.
    
    # If no features are left to branch or current_entropy is 0 (or nearly 0) then make a leaf
    # Entropy value of 0 means y label is perfectly one sided.
    if ((len(columns) == 0) | (current_ent < 0.01)):
        dec = df.iloc[:,0].value_counts().idxmax()
        return Node(None, dec)
    
    information_gain = [] # This would finally contain all the info gain from all the columns of the dataframe.
    
    for col in columns:
        if algo == 'id3':
            ig = info_gain(current_ent, col, df)
        elif algo == 'c4.5':
            ig = gain_ratio(current_ent, col, df)
            
        information_gain.append(ig)
    
    ind = np.argmax(information_gain)
    
    # This is the internal node of the tree
    tree_node = df.columns[ind+1]
    
    # The Node class object holds internal tree nodes that would be branching.
    # fit_dt() function is called recursively to construct/train the Decision Tree.
    
    i_node = Node(tree_node, None)
    branch_val = df[tree_node].unique()
    
    for brn in branch_val:        
        a = list(df.columns)
        b = tree_node        
        sub_col_list = [v for v in a if v != b]
        
        # Recursive calling of fit_dt() 
        sub_tree = fit_dt(df[df[tree_node] == brn][sub_col_list], algorithm = algo)
        # sub_tree = fit_dt(df_temp, algorithm = algo)     
        i_node.branches.append(sub_tree)
        i_node.branch_names.append(brn)
    
    return i_node


In [102]:
# Loading the training and testing dataset

# train = pd.read_csv('Data\\training.data', header = None)
train_0 = pd.read_csv('Data\\training_aa.data', header = None)
train_1 = pd.read_csv('Data\\training_ab.data', header = None)
train_2 = pd.read_csv('Data\\training_ac.data', header = None)
train_3 = pd.read_csv('Data\\training_ad.data', header = None)
train_4 = pd.read_csv('Data\\training_ae.data', header = None)
train_5 = pd.read_csv('Data\\training_af.data', header = None)
train_6 = pd.read_csv('Data\\training_ag.data', header = None)
train_7 = pd.read_csv('Data\\training_ah.data', header = None)
train_8 = pd.read_csv('Data\\training_ai.data', header = None)
train_9 = pd.read_csv('Data\\training_aj.data', header = None)
test = pd.read_csv('Data\\testing.data', header = None)


In [103]:
# Validation
train_list = [train_0, train_1, train_2, train_3, train_4, train_5, train_6, train_7, train_8, train_9]

def validation(t_list):    
    id3_max_score = 0
    c45_max_score = 0
    id3_best_tree = None
    c45_best_tree = None    
    
    n = len(t_list)    
    for i in range(n):
        val = t_list[i]  # The validation dataset
        del t_list[i]
        
        train = reduce(lambda x,y: pd.concat([x,y]), t_list) # Merging the remaining 9 training set
        id3_temp_tree = fit_dt(train, algorithm = 'id3') # Training the decision tree with the training data
        c45_temp_tree = fit_dt(train, algorithm = 'c4.5') # Training the decision tree with the training data
    
        id3_prediction = id3_temp_tree.predict(val) # Predicting on the validation dataset using the recent trained decision tree
        id3_score = met.f1_score(val[0].values, id3_prediction.values, pos_label = 'e') # F1 score computation
        
        c45_prediction = c45_temp_tree.predict(val) # Predicting on the validation dataset using the recent trained decision tree
        c45_score = met.f1_score(val[0].values, c45_prediction.values, pos_label = 'e') # F1 score computation
        
        if id3_score >= id3_max_score:
            id3_max_score = id3_score
            id3_best_tree = id3_temp_tree
            
        if c45_score >= c45_max_score:
            c45_max_score = c45_score
            c45_best_tree = c45_temp_tree
    
        train_list.insert(i, val) # Inserting back the validation set in training list for next round.

    print("Best id3 validation score is: ", id3_max_score)
    print("Best c4.53 validation score is: ", c45_max_score)
    
    return (id3_best_tree, c45_best_tree)

id3_tree, c45_tree = validation(train_list)


Best id3 validation score is:  1.0
Best c4.53 validation score is:  1.0


In [104]:
# Tree printing
print("The c45 tree: ")
print("---------------------")
c45_tree.print()

print("")
print("The id3 tree: ")
print("---------------------")
id3_tree.print()


The c45 tree: 
---------------------
-(Root)-> N:  5
   -(n)-> N:  20
      -(n)-> L:  e
      -(k)-> L:  e
      -(w)-> N:  17
         -(w)-> N:  8
            -(b)-> L:  e
            -(n)-> N:  7
               -(w)-> N:  4
                  -(f)-> L:  e
                  -(t)-> L:  p
               -(c)-> L:  p
         -(y)-> L:  p
      -(o)-> L:  e
      -(h)-> L:  e
      -(b)-> L:  e
      -(r)-> L:  p
      -(y)-> L:  e
   -(s)-> L:  p
   -(f)-> L:  p
   -(y)-> L:  p
   -(l)-> L:  e
   -(p)-> L:  p
   -(c)-> L:  p
   -(a)-> L:  e
   -(m)-> L:  p

The id3 tree: 
---------------------
-(Root)-> N:  5
   -(n)-> N:  20
      -(n)-> L:  e
      -(k)-> L:  e
      -(w)-> N:  22
         -(p)-> L:  e
         -(g)-> L:  e
         -(w)-> L:  e
         -(l)-> N:  3
            -(n)-> L:  e
            -(y)-> L:  p
            -(c)-> L:  e
            -(w)-> L:  p
         -(d)-> N:  8
            -(n)-> L:  p
            -(b)-> L:  e
      -(o)-> L:  e
      -(h)-> L:  e
      -(b)

In [105]:
# Prediction on testing dataset
id3_prediction = id3_tree.predict(test)
id3_score = met.f1_score(test[0].values, id3_prediction.values, pos_label = 'e') # F1 score computation

c45_prediction = c45_tree.predict(test)
c45_score = met.f1_score(test[0].values, c45_prediction.values, pos_label = 'e') # F1 score computation

print("F1 score on test dataset using id3 algorithm is: ", id3_score)
print("F1 score on test dataset using C4.5 algorithm is: ", c45_score)


F1 score on test dataset using id3 algorithm is:  1.0
F1 score on test dataset using C4.5 algorithm is:  1.0
